In [1]:
!nvidia-smi

Mon Dec  9 15:17:27 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A30                     Off | 00000000:17:00.0 Off |                    0 |
| N/A   26C    P0              30W / 165W |      9MiB / 24576MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [4]:
!pip install -Uqqq pip --progress-bar off
# !pip install -qqq torch==2.2.0 --progress-bar off
# !pip install -qqq transformers==4.37.2 --progress-bar off
# !pip install -qqq datasets==2.16.1 --progress-bar off
!pip install -qqq accelerate==1.2.0 --progress-bar off
!pip install -qqq bitsandbytes==0.45.0 --progress-bar off
!pip install -qqq peft==0.14.0 --progress-bar off
!pip install -qqq trl==0.12.2 --progress-bar off
!pip install -qqq flash-attn==2.7.2.post1 --progress-bar off
# !pip install -qqq --upgrade transformers --progress-bar off


## Data

In [6]:
import inspect
import random
import sqlite3

import matplotlib as mpl
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
from datasets import Dataset
from pylab import rcParams
from sklearn.metrics import confusion_matrix
from torch.utils.data import DataLoader
# from trl import DataCollatorForCompletionOnlyLM, SFTTrainer

%matplotlib inline
%config InlineBackend.figure_format='retina'

COLORS = ["#bae1ff", "#ffb3ba", "#ffdfba", "#ffffba", "#baffc9"]

sns.set(style="whitegrid", palette="muted", font_scale=1.2)
sns.set_palette(sns.color_palette(COLORS))

cmap = colors.LinearSegmentedColormap.from_list("custom_cmap", COLORS[:2])

MY_STYLE = {
    "figure.facecolor": "black",
    "axes.facecolor": "black",
    "axes.edgecolor": "white",
    "axes.labelcolor": "white",
    "axes.linewidth": 0.5,
    "text.color": "white",
    "xtick.color": "white",
    "ytick.color": "white",
    "grid.color": "gray",
    "grid.linestyle": "--",
    "grid.linewidth": 0.5,
    "axes.grid": True,
    "xtick.labelsize": "medium",
    "ytick.labelsize": "medium",
    "axes.titlesize": "large",
    "axes.labelsize": "large",
    "lines.color": COLORS[0],
    "patch.edgecolor": "white",
}

mpl.rcParams.update(MY_STYLE)
SEED = 42


def seed_everything(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)


seed_everything(SEED)

In [7]:
!pip install gdown
!gdown 1rlX7CjTBeFLz2lvjaM1RbFGOX9beALQz

  Using cached gdown-5.2.0-py3-none-any.whl.metadata (5.8 kB)
Using cached gdown-5.2.0-py3-none-any.whl (18 kB)
Downloading...
From: https://drive.google.com/uc?id=1rlX7CjTBeFLz2lvjaM1RbFGOX9beALQz
To: /home/dcn/tiny-llm/crypto-news-db.zip
100%|██████████████████████████████████████| 4.85M/4.85M [00:01<00:00, 3.99MB/s]


In [8]:
!unzip crypto-news-db.zip


Archive:  crypto-news-db.zip
  inflating: crypto-news.db          


In [9]:
connection = sqlite3.connect("crypto-news.db")
train_df = pd.read_sql_query("SELECT * FROM train", connection)
val_df = pd.read_sql_query("SELECT * FROM validation", connection)
test_df = pd.read_sql_query("SELECT * FROM test", connection)

train_df.shape, val_df.shape, test_df.shape

((24829, 9), (4966, 9), (1242, 9))

In [ ]:
train_df.head()

In [ ]:
def create_grid(n_rows, n_columns):
    fig, axes = plt.subplots(
        nrows=n_rows, ncols=n_columns, figsize=(n_columns * 5, n_rows * 4)
    )
    return fig, axes.flatten()

In [ ]:
fig, axes = create_grid(n_rows=3, n_columns=3)

subjects = [
    train_df.subject.value_counts(),
    val_df.subject.value_counts(),
    test_df.subject.value_counts(),
]
sentiments = [
    train_df.sentiment.value_counts(),
    val_df.sentiment.value_counts(),
    test_df.sentiment.value_counts(),
]
subjectivities = [train_df.subjectivity, val_df.subjectivity, test_df.subjectivity]

axes[0].bar(subjects[0].index, subjects[0].values)
axes[0].set_title("Subject")
axes[0].set_ylabel("Train")
axes[0].set_xticklabels([])
axes[0].set_xticks([])

axes[3].bar(subjects[1].index, subjects[1].values)
axes[3].set_ylabel("Validation")
axes[3].set_xticklabels([])
axes[3].set_xticks([])

axes[6].bar(subjects[2].index, subjects[2].values)
axes[6].set_ylabel("Test")
axes[6].set_xticks(axes[6].get_xticks())
axes[6].set_xticklabels(subjects[2].index, rotation=90)

axes[1].bar(sentiments[0].index, sentiments[0].values)
axes[1].set_title("Sentiment")
axes[1].set_xticklabels([])
axes[1].set_xticks([])

axes[4].bar(sentiments[1].index, sentiments[1].values)
axes[4].set_xticklabels([])
axes[4].set_xticks([])

axes[7].bar(sentiments[2].index, sentiments[2].values)
axes[7].set_xticks(axes[7].get_xticks())
axes[7].set_xticklabels(sentiments[2].index, rotation=90)

axes[2].hist(subjectivities[0])
axes[2].set_title("Subjectivity")
axes[2].set_xticklabels([])
axes[2].set_xticks([])

axes[5].hist(subjectivities[1])
axes[5].set_xticklabels([])
axes[5].set_xticks([])

axes[8].hist(subjectivities[2])

plt.show();

In [ ]:
dataset = {
    "train": Dataset.from_pandas(train_df),
    "val": Dataset.from_pandas(val_df),
    "test": Dataset.from_pandas(test_df),
}
dataset

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

MODEL_A_TINYLLAMA = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"
#MODEL_B_DEEPSEEK = "deepseek-ai/DeepSeek-V2.5"
MODEL_B_Phi_3 = "microsoft/Phi-3-mini-4k-instruct"
MODEL_C_ROBERTA = "FacebookAI/roberta-base"
MODEL_D_distilbert = "distilbert/distilbert-base-uncased"

## Initialize the models

### TinyLLAMA

In [ ]:

PAD_TOKEN = "<pad>"

tokenizer_tiny_llama = AutoTokenizer.from_pretrained(MODEL_A_TINYLLAMA, add_eos_token=True, use_fast=True)
tokenizer_tiny_llama.add_special_tokens({"pad_token": PAD_TOKEN})
tokenizer_tiny_llama.padding_side = "right"

model_tiny_llama = AutoModelForCausalLM.from_pretrained(
    MODEL_A_TINYLLAMA,
    device_map="auto",
    trust_remote_code=True,
    # attn_implementation="flash_attention_2" if available
)
model_tiny_llama.resize_token_embeddings(len(tokenizer_tiny_llama), pad_to_multiple_of=8)

In [ ]:
#In the context of Hugging Face's tokenizers, the bos_token and bos_token_id refer to the Beginning of Sequence (BOS) token and its corresponding ID in the tokenizer's vocabulary.
tokenizer_tiny_llama.bos_token, tokenizer_tiny_llama.bos_token_id

In [ ]:
tokenizer_tiny_llama.eos_token, tokenizer_tiny_llama.eos_token_id

In [ ]:
tokenizer_tiny_llama.pad_token, tokenizer_tiny_llama.pad_token_id

In [ ]:
#Check if PAD_TOKEN Exists: If PAD_TOKEN (like "<pad>") has been added to the tokenizer using tokenizer.add_special_tokens, it should appear in the vocabulary.
PAD_TOKEN in tokenizer_tiny_llama.get_vocab()

### Phi 3

In [ ]:
tokenizer_phi3 = AutoTokenizer.from_pretrained(MODEL_B_Phi_3, add_eos_token=True, use_fast=True)
tokenizer_phi3.add_special_tokens({"pad_token": PAD_TOKEN})
tokenizer_phi3.padding_side = "right"

model_phi3 = AutoModelForCausalLM.from_pretrained(
    MODEL_B_Phi_3,
    device_map="auto",
    trust_remote_code=True,
    # attn_implementation="flash_attention_2" if available
)
model_phi3.resize_token_embeddings(len(tokenizer_phi3), pad_to_multiple_of=8)

### roberta-base

In [ ]:
tokenizer_roberta_base = AutoTokenizer.from_pretrained(MODEL_C_ROBERTA, add_eos_token=True, use_fast=True)
tokenizer_roberta_base.add_special_tokens({"pad_token": PAD_TOKEN})
tokenizer_roberta_base.padding_side = "right"

model_roberta_base = AutoModelForCausalLM.from_pretrained(
    MODEL_C_ROBERTA,
    device_map="auto",
    trust_remote_code=True,
    # attn_implementation="flash_attention_2" if available
)
model_roberta_base.resize_token_embeddings(len(tokenizer_roberta_base), pad_to_multiple_of=8)

### distilbert-base-uncased

In [ ]:
from transformers import AutoModelForSequenceClassification

tokenizer_distilbert = AutoTokenizer.from_pretrained(MODEL_D_distilbert, add_eos_token=True, use_fast=True)


# model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncase", num_labels=3)

model_distilbert = AutoModelForSequenceClassification.from_pretrained(
    MODEL_D_distilbert,
    # device_map="auto",
    # trust_remote_code=True,
    # attn_implementation="flash_attention_2" if available
)
model_distilbert.resize_token_embeddings(len(tokenizer_distilbert), pad_to_multiple_of=8)

## Token Counts

In [ ]:
#template used

def format_example(example):
    return inspect.cleandoc(
        f"""
### Title:
{example["title"]}
### Text:
{example["text"]}
### Prediction:
subject: {example["subject"]}
sentiment: {example["sentiment"]}
"""
    )

In [ ]:
print(format_example(dataset["train"][0]))

In [ ]:
print(type(tokenizer_roberta_base), tokenizer_roberta_base)

### TinyLLama token counts

In [ ]:
#count the number of tokens in each example, finding the number of tokens we are going to need.
#we have a limit of 2048, so the number of tokens we are going to need is 200.
from tqdm import tqdm

token_counts_tiny_llama = []
# Compute token counts for tiny_llama tokenizer
for example in tqdm(dataset["train"], desc="Processing Tiny LLama"):
    token_counts_tiny_llama.append(
        len(
            tokenizer_tiny_llama(
                format_example(example),
                add_special_tokens=True,
                return_attention_mask=False,
            )["input_ids"]
        )
    )

token_counts_phi3 = []
# Compute token counts for phi3 tokenizer
for example in tqdm(dataset["train"], desc="Processing Phi3"):
    token_counts_phi3.append(
        len(
            tokenizer_phi3(
                format_example(example),
                add_special_tokens=True,
                return_attention_mask=False,
            )["input_ids"]
        )
    )

token_counts_roberta = []
# Compute token counts for roberta-base tokenizer
for example in tqdm(dataset["train"], desc="Processing Roberta Base"):
    token_counts_roberta.append(
        len(
            tokenizer_roberta_base(
                format_example(example),
                add_special_tokens=True,
                return_attention_mask=False,
            )["input_ids"]
        )
    )

token_counts_distilbert = []
# Compute token counts for roberta-base tokenizer
for example in tqdm(dataset["train"], desc="Processing DistilBert Base"):
    token_counts_distilbert.append(
        len(
            tokenizer_distilbert(
                format_example(example),
                add_special_tokens=True,
                return_attention_mask=False,
            )["input_ids"]
        )
    )


# Plot histograms for both tokenizers
plt.hist(token_counts_tiny_llama, bins=30, alpha=0.7, label="Tiny LLama")
plt.hist(token_counts_phi3, bins=30, alpha=0.7, label="Phi3")
plt.hist(token_counts_roberta, bins=30, alpha=0.7, label="Roberta")
plt.hist(token_counts_distilbert, bins=30, alpha=0.7, label="DistilBert")
plt.axvline(200, color="red", linestyle="--", label="Token Limit (200)")
plt.xlabel("Number of Tokens")
plt.ylabel("Frequency")
plt.title("Token Distribution")
plt.legend()
plt.show()

# Training

### TinyLLama

In [ ]:
model_tiny_llama.pad_token_id = tokenizer_tiny_llama.pad_token_id
model_tiny_llama.config.pad_token_id = tokenizer_tiny_llama.pad_token_id

In [ ]:
model_tiny_llama.config

In [ ]:
# check the model to see where we need to apply the LoRa scaling or the LoRa target modules
# self attention and mlp layers
model_tiny_llama

## Phi 3

In [ ]:
model_phi3.pad_token_id = tokenizer_phi3.pad_token_id
model_phi3.config.pad_token_id = tokenizer_phi3.pad_token_id

In [ ]:

model_phi3.config

In [ ]:
model_phi3

### roberta base

In [ ]:

model_roberta_base.pad_token_id = tokenizer_roberta_base.pad_token_id
model_roberta_base.config.pad_token_id = tokenizer_roberta_base.pad_token_id

In [ ]:
model_roberta_base.config

In [ ]:
model_roberta_base

### distilbert

In [ ]:
model_distilbert.pad_token_id = tokenizer_distilbert.pad_token_id
model_distilbert.config.pad_token_id = tokenizer_distilbert.pad_token_id

In [ ]:
model_distilbert.config

In [ ]:
model_distilbert

### LoRA Setup

#### RECALL

Fine-tuning with PEFT (Parameter-Efficient Fine-Tuning) is a method to adapt large language models (LLMs) for specific tasks without training the entire model. It works by freezing the LLM's weights and adding a few trainable parameters (like adapters) to learn the task-specific information

#### TinyLLAMA Lora Config

In [ ]:
from peft import LoraConfig, TaskType, get_peft_model

lora_config_tiny_llama = LoraConfig(
    r=128, # LoRa ranking of the metrics
    lora_alpha=128, # for scaling the parameters of LoRa
    target_modules=[
        "self_attn.q_proj",
        "self_attn.k_proj",
        "self_attn.v_proj",
        "self_attn.o_proj",
        "mlp.gate_proj",
        "mlp.up_proj",
        "mlp.down_proj",
    ],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

model_tiny_llama = get_peft_model(model_tiny_llama, lora_config_tiny_llama) # get the peft_model on top of original model and adding the LoRa config

model_tiny_llama.print_trainable_parameters()

#### Phi3 Lora Config

In [ ]:
lora_config_phi3 = LoraConfig(
    r=128, # LoRa ranking of the metrics
    lora_alpha=128, # for scaling the parameters of LoRa
    target_modules=[
        "self_attn.o_proj",
        "self_attn.qkv_proj",
        
        "mlp.gate_up_proj",
        "mlp.down_proj",
    ],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

model_phi3 = get_peft_model(model_phi3, lora_config_phi3) # get the peft_model on top of original model and adding the LoRa config

model_phi3.print_trainable_parameters()

### Roberta  Lora Config 

In [ ]:
lora_config_roberta = LoraConfig(
    r=128, # LoRa ranking of the metrics
    lora_alpha=128, # for scaling the parameters of LoRa
    target_modules=[
         "attention.self.query",
            "attention.self.value",
            "attention.output.dense",
            "intermediate.dense",
            "output.dense",
    ],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

model_roberta_base = get_peft_model(model_roberta_base, lora_config_roberta) # get the peft_model on top of original model and adding the LoRa config

model_roberta_base.print_trainable_parameters()

### Distilbert  Lora Config

In [ ]:
lora_config_distilbert = LoraConfig(
    r=128, # LoRa ranking of the metrics
    lora_alpha=128, # for scaling the parameters of LoRa
    target_modules=[
        # Attention layer
        "transformer.layer.*.attention.q_lin",  # Query projection
        "transformer.layer.*.attention.k_lin",  # Key projection
        "transformer.layer.*.attention.v_lin",  # Value projection
        "transformer.layer.*.attention.out_lin",  # Output projection after attention
        
        # Feed-forward network
        "transformer.layer.*.ffn.lin1",  # First linear layer in FFN (expansion)
        "transformer.layer.*.ffn.lin2",  # Second linear layer in FFN (contraction)
        
        # Classification head (optional, depending on task)
        "pre_classifier",  # Classification projection before the final layer
        "classifier",  # Final classification layer
    ],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

model_distilbert = get_peft_model(model_distilbert, lora_config_distilbert) # get the peft_model on top of original model and adding the LoRa config

model_distilbert.print_trainable_parameters()

# Training on Completions

### TinyLLama

In [ ]:
response_template = "\n### Prediction:"
response_template_ids_tiny_llama = tokenizer_tiny_llama.encode(response_template, add_special_tokens=False)[
    2:
]
collator_tiny_llama = DataCollatorForCompletionOnlyLM(response_template_ids_tiny_llama, tokenizer=tokenizer_tiny_llama)

examples = [format_example(dataset["train"][0])]
encodings_tiny_llama = [tokenizer_tiny_llama(e) for e in examples]

dataloader_tiny_llama = DataLoader(encodings_tiny_llama, collate_fn=collator_tiny_llama, batch_size=1)

In [ ]:
batch_tiny_llama = next(iter(dataloader_tiny_llama))
batch_tiny_llama.keys()

In [ ]:
batch_tiny_llama["labels"]

### Phi3

In [ ]:
response_template_ids_phi3 = tokenizer_phi3.encode(response_template, add_special_tokens=False)[
    2:
]
collator_phi3 = DataCollatorForCompletionOnlyLM(response_template_ids_phi3, tokenizer=tokenizer_phi3)

encodings_phi3 = [tokenizer_phi3(e) for e in examples]

dataloader_phi3 = DataLoader(encodings_phi3, collate_fn=collator_phi3, batch_size=1)

In [ ]:
batch_phi3 = next(iter(dataloader_phi3))
batch_phi3.keys()

In [ ]:
batch_phi3["labels"]

### Roberta

In [ ]:
response_template_ids_roberta_base = tokenizer_roberta_base.encode(response_template, add_special_tokens=False)[
    2:
]
collator_roberta_base = DataCollatorForCompletionOnlyLM(response_template_ids_roberta_base, tokenizer=tokenizer_roberta_base)

encodings_roberta_base = [tokenizer_roberta_base(e) for e in examples]

dataloader_roberta_base = DataLoader(encodings_roberta_base, collate_fn=collator_roberta_base, batch_size=1)

In [ ]:
batch_roberta_base = next(iter(dataloader_roberta_base))
batch_roberta_base.keys()

In [ ]:
batch_roberta_base["labels"]

### Distilbert

In [ ]:
response_template_ids_distilbert = tokenizer_distilbert.encode(response_template, add_special_tokens=False)[
    2:
]
collator_distilbert = DataCollatorForCompletionOnlyLM(response_template_ids_distilbert, tokenizer=tokenizer_distilbert)

encodings_distilbert = [tokenizer_distilbert(e) for e in examples]

dataloader_distilbert = DataLoader(encodings_distilbert, collate_fn=collator_distilbert, batch_size=1)

In [ ]:
batch_distilbert = next(iter(dataloader_distilbert))
batch_distilbert.keys()

# Training Arguments

### TinyLlama

In [ ]:
!pip install -Uqqq tensorboard  --progress-bar off


In [ ]:
from transformers import TrainingArguments

training_arguments_tiny_llama = TrainingArguments(
    output_dir="experiments",
    num_train_epochs=1,
    per_device_train_batch_size=8, # pass four examples to the GPU
    gradient_accumulation_steps=8, # accumulate on top of the batch
    optim="adamw_torch",
    evaluation_strategy="steps",
    eval_steps=0.2,
    logging_steps=10,
    learning_rate=1e-4,
    fp16=True,  # or bf16=True, quantization is not used to keep the model smaller and faster
    save_strategy="epoch",
    max_grad_norm=1.0,
    warmup_ratio=0.1,
    lr_scheduler_type="constant",
    report_to="tensorboard",
    save_safetensors=True,
    seed=SEED,
)

In [ ]:
def format_prompts(example):
    output_texts = []
    for i in range(len(example["title"])):
        text = inspect.cleandoc(
            f"""
### Title:
{example["title"][i]}
### Text:
{example["text"][i]}
### Prediction:
subject: {example["subject"][i]}
sentiment: {example["sentiment"][i]}
"""
        )
        output_texts.append(text)
    return output_texts

In [ ]:
from trl import SFTTrainer

trainer_tiny_llama = SFTTrainer(
    model=model_tiny_llama,
    args=training_arguments_tiny_llama,
    train_dataset=dataset["train"],
    eval_dataset=dataset["val"],
    tokenizer=tokenizer_tiny_llama,
    max_seq_length=1024,
    formatting_func=format_prompts,
    data_collator=collator_tiny_llama,
)

In [ ]:
# %reload_ext tensorboard
# %tensorboard --logdir experiments/runs
# %tensorboard --logdir logs/fit

# !kill 40572

In [ ]:
trainer_tiny_llama.train()

In [ ]:
trainer_tiny_llama.model.save_pretrained("tiny-llama-crypto-sentiment-model")
tokenizer_tiny_llama.save_pretrained("tiny-llama-crypto-sentiment-model")

### Phi3

In [ ]:
training_arguments_phi3 = TrainingArguments(
    output_dir="experiments",
    num_train_epochs=1,
    per_device_train_batch_size=4, # pass four examples to the GPU
    gradient_accumulation_steps=4, # accumulate on top of the batch
    optim="adamw_torch",
    evaluation_strategy="steps",
    eval_steps=0.2,
    logging_steps=10,
    learning_rate=1e-4,
    fp16=True,  # or bf16=True, quantization is not used to keep the model smaller and faster
    save_strategy="epoch",
    max_grad_norm=1.0,
    warmup_ratio=0.1,
    lr_scheduler_type="constant",
    report_to="tensorboard",
    save_safetensors=True,
    seed=SEED,
)

In [ ]:
trainer_phi3 = SFTTrainer(
    model=model_phi3,
    args=training_arguments_phi3,
    train_dataset=dataset["train"],
    eval_dataset=dataset["val"],
    tokenizer=tokenizer_phi3,
    max_seq_length=1024,
    formatting_func=format_prompts,
    data_collator=collator_phi3,
)

In [ ]:
trainer_phi3.train()

In [ ]:
trainer_phi3.model.save_pretrained("phi3-crypto-sentiment-model")
tokenizer_phi3.save_pretrained("phi3-crypto-sentiment-model")

### Roberta

In [ ]:
lora_config_roberta = LoraConfig(
    r=6, # LoRa ranking of the metrics
    lora_alpha=8, # for scaling the parameters of LoRa
    target_modules=[
         "attention.self.query",
            "attention.self.value",
            "attention.output.dense",
            "intermediate.dense",
            "output.dense",
    ],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

model_roberta_base = get_peft_model(model_roberta_base, lora_config_roberta) # get the peft_model on top of original model and adding the LoRa config

model_roberta_base.print_trainable_parameters()

In [ ]:
training_arguments_roberta_base = TrainingArguments(
    output_dir="experiments",
    num_train_epochs=1,
    per_device_train_batch_size=8, # pass four examples to the GPU
    gradient_accumulation_steps=8, # accumulate on top of the batch
    optim="adamw_torch",
    evaluation_strategy="steps",
    eval_steps=0.2,
    logging_steps=10,
    learning_rate=1e-4,
    fp16=True,  # or bf16=True, quantization is not used to keep the model smaller and faster
    save_strategy="epoch",
    max_grad_norm=1.0,
    warmup_ratio=0.1,
    lr_scheduler_type="constant",
    report_to="tensorboard",
    save_safetensors=True,
    seed=SEED,
)

In [ ]:
trainer_roberta_base = SFTTrainer(
    model=model_roberta_base,
    args=training_arguments_roberta_base,
    train_dataset=dataset["train"],
    eval_dataset=dataset["val"],
    tokenizer=tokenizer_roberta_base,
    max_seq_length=256,
    formatting_func=format_prompts,
    data_collator=collator_roberta_base,
)

In [ ]:
trainer_roberta_base.train()

In [ ]:
trainer_roberta_base.model.save_pretrained("roberta-base-crypto-sentiment-model")
tokenizer_roberta_base.save_pretrained("roberta-base-crypto-sentiment-model")

# Load Trained Model TinyLLAMA

In [ ]:
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

MODEL_NAME_TINY_LLAMA = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"

base_model_tiny_llama = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME_TINY_LLAMA,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True,
)

base_model_tiny_llama.resize_token_embeddings(len(tokenizer_tiny_llama), pad_to_multiple_of=8)

model_new_tiny_llama = PeftModel.from_pretrained(base_model_tiny_llama, "tiny-llama-crypto-sentiment-model")
model_new_tiny_llama = model_new_tiny_llama.merge_and_unload()

tokenizer_tiny_llama = AutoTokenizer.from_pretrained(
    "tiny-llama-crypto-sentiment-model", trust_remote_code=True
)

tokenizer_tiny_llama.pad_token, tokenizer_tiny_llama.pad_token_id, tokenizer_tiny_llama.padding_side

In [ ]:
model_new_tiny_llama.pad_token_id = tokenizer_tiny_llama.pad_token_id
model_new_tiny_llama.config.pad_token_id = tokenizer_tiny_llama.pad_token_id

## Evaluation

In [ ]:
from transformers import pipeline

pipe_tiny_llama = pipeline(
    task="text-generation",
    model=model_new_tiny_llama,
    tokenizer=tokenizer_tiny_llama,
    max_new_tokens=16,
    return_full_text=False,
)

In [ ]:
def format_for_prediction(example):
    return inspect.cleandoc(
        f"""
### Title:
{example["title"]}
### Text:
{example["text"]}
### Prediction:
"""
    )

In [ ]:
from transformers import logging

logging.set_verbosity(logging.CRITICAL)

In [ ]:
for i in range(10):
    example = dataset["test"][i]
    print(f"Example {i + 1}\n")
    text = inspect.cleandoc(
        f"""
        Title: {example["title"]}
        Text: {example["text"]}
        Subject: {example["subject"]}
        Sentiment: {example["sentiment"]}
    """
    )
    print(text)
    print("\nPrediction:")
    outputs_tiny_llama = pipe_tiny_llama(format_for_prediction(example))
    print(outputs_tiny_llama[0]["generated_text"])
    print()

In [ ]:
import re

from tqdm import tqdm

rows = []

for i in tqdm(range(len(dataset["test"]))):
    example = dataset["test"][i]
    outputs_tiny_llama = pipe_tiny_llama(format_for_prediction(example))
    text = outputs_tiny_llama[0]["generated_text"]
    parts = re.split(r"[:\n]", text.strip())
    subject, sentiment = parts[1].strip(), parts[3].strip()

    rows.append(
        {
            "title": example["title"],
            "text": example["text"],
            "true_subject": example["subject"],
            "true_sentiment": example["sentiment"],
            "pred_subject": subject,
            "pred_sentiment": sentiment,
        }
    )

In [ ]:
predictions_df = pd.DataFrame(rows)
predictions_df.head()

In [ ]:
(predictions_df["true_subject"] == predictions_df["pred_subject"]).sum() / len(
    predictions_df
)

In [ ]:
labels = predictions_df["true_subject"].unique().tolist()
cf_matrix = confusion_matrix(
    predictions_df["true_subject"], predictions_df["pred_subject"], labels=labels
)
_, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(
    cf_matrix,
    cmap=cmap,
    ax=ax,
)
ax.set_yticklabels(labels, rotation=0)
ax.set_xticklabels(labels, rotation=45);

In [ ]:
predictions_df[predictions_df["true_subject"] != predictions_df["pred_subject"]].head(
    n=10
)[["title", "text", "true_subject", "pred_subject"]]

In [ ]:
(predictions_df["true_sentiment"] == predictions_df["pred_sentiment"]).sum() / len(
    predictions_df
)

In [ ]:
labels = predictions_df["true_sentiment"].unique().tolist()
cf_matrix = confusion_matrix(
    predictions_df["true_sentiment"], predictions_df["pred_sentiment"], labels=labels
)
_, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(
    cf_matrix,
    cmap=cmap,
    ax=ax,
)
ax.set_yticklabels(labels, rotation=0)
ax.set_xticklabels(labels, rotation=45);

In [ ]:
predictions_df[
    predictions_df["true_sentiment"] != predictions_df["pred_sentiment"]
].head(n=10)[["title", "text", "true_sentiment", "pred_sentiment"]]

# Load Trained Model Phi3

In [ ]:
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

# MODEL_B_Phi_3 = "microsoft/Phi-3-mini-4k-instruct"
# MODEL_C_ROBERTA = "FacebookAI/roberta-base"
# MODEL_D_distilbert = "distilbert/distilbert-base-uncased"

MODEL_NAME_phi3 = MODEL_B_Phi_3

base_model_phi3 = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME_phi3,
    # device_map="auto",
    # torch_dtype=torch.float16,
    trust_remote_code=True,
)

base_model_phi3.resize_token_embeddings(len(tokenizer_phi3), pad_to_multiple_of=8)

model_new_phi3 = PeftModel.from_pretrained(base_model_phi3, "phi3-crypto-sentiment-model")
model_new_phi3 = model_new_phi3.merge_and_unload()

tokenizer_phi3 = AutoTokenizer.from_pretrained(
    "phi3-crypto-sentiment-model", trust_remote_code=True
)

tokenizer_phi3.pad_token, tokenizer_phi3.pad_token_id, tokenizer_phi3.padding_side

## Evaluation

In [ ]:
pipe_phi3 = pipeline(
    task="text-generation",
    model=model_new_phi3,
    tokenizer=tokenizer_phi3,
    max_new_tokens=16,
    return_full_text=False,
)

In [ ]:
for i in range(5):
    example = dataset["test"][i]
    print(f"Example {i + 1}\n")
    text = inspect.cleandoc(
        f"""
        Title: {example["title"]}
        Text: {example["text"]}
        Subject: {example["subject"]}
        Sentiment: {example["sentiment"]}
    """
    )
    print(text)
    print("\nPrediction:")
    outputs_phi3 = pipe_phi3(format_for_prediction(example))
    print(outputs_phi3[0]["generated_text"])
    print()

In [ ]:
rows = []

for i in tqdm(range(len(dataset["test"]))):
    example = dataset["test"][i]
    outputs_phi3 = pipe_phi3(format_for_prediction(example))
    text = outputs_phi3[0]["generated_text"]
    parts = re.split(r"[:\n]", text.strip())
    subject, sentiment = parts[1].strip(), parts[3].strip()

    rows.append(
        {
            "title": example["title"],
            "text": example["text"],
            "true_subject": example["subject"],
            "true_sentiment": example["sentiment"],
            "pred_subject": subject,
            "pred_sentiment": sentiment,
        }
    )

In [ ]:
predictions_df = pd.DataFrame(rows)
predictions_df.head()

In [ ]:
(predictions_df["true_subject"] == predictions_df["pred_subject"]).sum() / len(
    predictions_df
)

In [ ]:
predictions_df[predictions_df["true_subject"] != predictions_df["pred_subject"]].head(
    n=10
)[["title", "text", "true_subject", "pred_subject"]]

In [ ]:
(predictions_df["true_sentiment"] == predictions_df["pred_sentiment"]).sum() / len(
    predictions_df
)

# Load Trained Model Roberta

In [ ]:

MODEL_NAME_ROBERTA_BASE = MODEL_C_ROBERTA

base_model_roberta_base = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME_ROBERTA_BASE,
    # device_map="auto",
    # torch_dtype=torch.float16,
    trust_remote_code=True,
)

base_model_roberta_base.resize_token_embeddings(len(tokenizer_roberta_base), pad_to_multiple_of=8)

model_new_roberta_base = PeftModel.from_pretrained(base_model_roberta_base, "roberta_base-crypto-sentiment-model")
model_new_roberta_base = model_roberta_base.merge_and_unload()

tokenizer_phi3 = AutoTokenizer.from_pretrained(
    "phi3-crypto-sentiment-model", trust_remote_code=True
)

tokenizer_phi3.pad_token, tokenizer_phi3.pad_token_id, tokenizer_phi3.padding_side

### Evaluation

In [ ]:
pipe_roberta_base = pipeline(
    task="text-generation",
    model=base_model_roberta_base,
    tokenizer=tokenizer_roberta_base,
    max_new_tokens=16,
    return_full_text=False,
)

In [ ]:
for i in range(5):
    example = dataset["test"][i]
    print(f"Example {i + 1}\n")
    text = inspect.cleandoc(
        f"""
        Title: {example["title"]}
        Text: {example["text"]}
        Subject: {example["subject"]}
        Sentiment: {example["sentiment"]}
    """
    )
    print(text)
    print("\nPrediction:")
    outputs_roberta_base = pipe_roberta_base(format_for_prediction(example))
    print(outputs_roberta_base[0]["generated_text"])
    print()

In [ ]:
rows = []

for i in tqdm(range(len(dataset["test"]))):
    example = dataset["test"][i]
    outputs_roberta_base = pipe_roberta_base(format_for_prediction(example))
    text = outputs_roberta_base[0]["generated_text"]
    print(outputs_roberta_base)
    parts = re.split(r"[:\n]", text.strip())
    subject, sentiment = parts[1].strip(), parts[3].strip()

    rows.append(
        {
            "title": example["title"],
            "text": example["text"],
            "true_subject": example["subject"],
            "true_sentiment": example["sentiment"],
            "pred_subject": subject,
            "pred_sentiment": sentiment,
        }
    )